# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
csv_file = os.path.join('..', 'output_data', 'cities.csv')

city_df = pd.read_csv(csv_file)
city_df

,Unnamed: 0,City,Lat,Lng,MaxTemp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,276.96,81,20,2.57,AR,1650173169
1,1,rikitea,-23.1203,-134.9692,299.09,67,17,6.57,PF,1650173229
2,2,new norfolk,-42.7826,147.0587,299.22,34,100,2.24,AU,1650173230
3,3,fortuna,40.5982,-124.1573,279.07,88,0,2.06,US,1650173230
4,4,port elizabeth,-33.9180,25.5701,289.44,95,20,4.12,ZA,1650173231
...,...,...,...,...,...,...,...,...,...,...
580,580,oktyabrskiy,54.4815,53.4710,279.59,63,95,4.38,RU,1650173682
581,581,sechura,-5.5569,-80.8222,292.94,80,15,5.59,PE,1650173683
582,582,comonfort,20.7167,-100.7667,297.15,52,92,2.19,MX,1650173684
583,583,ust-nera,64.5667,143.2000,268.99,71,100,0.35,RU,1650173684


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)

In [6]:
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
city_df['MaxTemp'] = (city_df['MaxTemp'] - 273.15)*1.8 + 32
ideal_df = city_df[(city_df['MaxTemp'] >= 70) & (city_df['MaxTemp'] <= 80)]
ideal_df = ideal_df[ideal_df['Wind Speed'] <= 10]
ideal_df = ideal_df[ideal_df['Cloudiness'] == 0]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = ideal_df
hotel_df['Hotel Name'] = ""
hotel_df

,Unnamed: 0,City,Lat,Lng,MaxTemp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
81,81,taoudenni,22.6783,-3.9836,70.520,32,0,4.45,ML,1650173289,
98,98,vao,-22.6667,167.4833,77.648,67,0,2.87,NC,1650173301,
155,155,dubbo,-32.2500,148.6167,77.126,38,0,3.09,AU,1650173343,
236,236,buraydah,26.3260,43.9750,73.850,10,0,5.56,SA,1650173411,
266,266,marsh harbour,26.5412,-77.0636,75.362,76,0,4.55,BS,1650173436,
303,303,adrar,20.5022,-10.0711,71.870,15,0,5.27,MR,1650173464,
316,316,aswan,24.0934,32.9070,78.098,13,0,3.09,EG,1650173341,
337,337,arona,28.0996,-16.6810,72.122,19,0,8.75,ES,1650173490,
390,390,vila velha,-20.3297,-40.2925,71.942,76,0,2.68,BR,1650173532,


In [9]:
for index, row in hotel_df.iterrows():
    try:
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }
        
        lat = row['Lat']
        lng = row['Lng']
        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df        

,Unnamed: 0,City,Lat,Lng,MaxTemp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
81,81,taoudenni,22.6783,-3.9836,70.520,32,0,4.45,ML,1650173289,NaN
98,98,vao,-22.6667,167.4833,77.648,67,0,2.87,NC,1650173301,OURE TERA beach resort
155,155,dubbo,-32.2500,148.6167,77.126,38,0,3.09,AU,1650173343,Garden Hotel
236,236,buraydah,26.3260,43.9750,73.850,10,0,5.56,SA,1650173411,Best Western Plus Buraidah
266,266,marsh harbour,26.5412,-77.0636,75.362,76,0,4.55,BS,1650173436,Abaco Beach Resort
303,303,adrar,20.5022,-10.0711,71.870,15,0,5.27,MR,1650173464,NaN
316,316,aswan,24.0934,32.9070,78.098,13,0,3.09,EG,1650173341,Pyramisa Island Hotel Aswan
337,337,arona,28.0996,-16.6810,72.122,19,0,8.75,ES,1650173490,Hotel Vanilla Garden
390,390,vila velha,-20.3297,-40.2925,71.942,76,0,2.68,BR,1650173532,Hotel Golden Tulip Porto Vitória


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

fig.add_layer(markers)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))